1\. Write a function that converts number representation, bin<->dec<->hex. (Clearly using the corresponding python built-in functions is not fair..)

In [6]:
def convert(num, from_base, to_base):
    digits='0123456789ABCDEF'
    num= num.upper()
    if from_base == "dec":
        dec= int(num)
    elif from_base == "bin":
        dec=0
        for i, bit in enumerate(num[::-1]):
            if bit not in '01':
                raise ValueError(f'Carattere non valido nel binario:{bit}')
            dec+= int(bit) *(2**i)
    elif from_base=='hex':
        dec=0
        for i, ch in enumerate(num[::-1]):
            if ch not in digits:
                raise ValueError(f"Carattere non valido nell'esadecimale: {ch}")
            dec+=digits.index(ch)* (16**i) #digits.index converte la lettera in un valore numerico 
    else:
        raise ErrorValue('base di partenza non valida')

    if to_base =='dec':
        return ( str(dec))
    base=2 if to_base == 'bin' else 16
    if dec==0:
        return '0'

    converted = ''
    while dec >0 :
        converted = digits [dec % base] + converted 
        dec //= base 
    return converted
  
print(convert("1011", "bin", "dec"))  # → 11
print(convert("1011", "bin", "hex"))  # → B
print(convert("255", "dec", "bin"))   # → 11111111
print(convert("255", "dec", "hex"))   # → FF
print(convert("2A", "hex", "dec"))    # → 42
print(convert("2A", "hex", "bin"))    # → 101010
print(convert("0", "dec", "hex"))     # → 0

        

11
B
11111111
FF
42
101010
0


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [11]:
def float_from32bit(word):
    if len(word) != 32:
        raise ValueError( 'La parola deve essere lunga 32 bit') 
    sign = int( word[0])
    s= (-1) ** sign 

    exponent_bits = word [1:9]
    exponent = 0
    for i, bit in enumerate(exponent_bits[::-1]):
        exponent += int(bit) * (2**i)
    exponent -=127 #bias

    mantissa_bits= word[9:]
    mantissa=1
    for i, bir in enumerate(mantissa_bits):
        mantissa +=int(bit) * (2** -(i+1))

    return s* mantissa * ( 2** exponent) 

word = '00111111100000000000000000000000'
print(float_from32bit(word)) #-> 1.0 

1.0


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [15]:
#overflow
x = 1.0 
while x != float('inf'):
    last_x=x
    x*=2
print ( f'overflow limit(approx):{last_x}')

#underflow
y= 1.0
while y != 0.0:
    last_y= y 
    y/=2
print( f'underflow limit( approx):{last_y}')

overflow limit(approx):8.98846567431158e+307
underflow limit( approx):5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [16]:
var = 1.0 
while True:
    test= 1.0 +var
    if test == 1.0: #se il risultato non cambia abbiamo superato la precisione macchina  
        break 
    var /=2 #riduciamo variable di metà per avvicinarsi sempre di piu al limite 

machine_variable= var*2  #alla fine questo valore è l'ultimo che ha ancora effetto su 1.0 
print('machine precision:', machine_variable)

machine precision: 2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [17]:
#a)
import numpy as np 

def function( a, b ,c ):
    x1= (-b + np.sqrt( b**2 - 4*a*c))/(2*a)
    x2= (-b - np.sqrt( b**2 - 4*a*c))/(2*a)
    return x1, x2
function(0.001, 1000, 0.001)

(np.float64(-9.999894245993346e-07), np.float64(-999999.999999))

In [1]:
#b)
import numpy as np 

def function( a, b ,c ):
    x1= (-2* c)/(b+ np.sqrt(b**2 - 4 *a*c))
    x2= (-2* c)/(b- np.sqrt(b**2 - 4 *a*c))
    return x1, x2
function(0.001, 1000, 0.001)

#Using the standard formula (part a), the smaller root (x1) 
#is inaccurate or even zero, while the larger root (x2) is correct.
#This happens due to catastrophic cancellation, because 
#-b + sqrt(b² - 4ac) subtracts two nearly equal large numbers.
#Using the rewritten formula (part b), both roots are calculated correctly.

(np.float64(-1.000000000001e-06), np.float64(-1000010.5755125057))

In [2]:
#c)
import numpy as np 

def quadratic(a, b , c):
    D= b**2 - 4*a*c
    if b>=0:
        x2= (-b -np.sqrt(D))/(2*a)
        x1= (2*c)/(-b -np.sqrt(D))
    else:
        x2= (-b +np.sqrtD)/(2*a)
        x1= (2*c)/(-b +np.sqrt(D))
    return x1, x2

quadratic( 0.001, 1000, 0.001) 

(np.float64(-1.000000000001e-06), np.float64(-999999.999999))

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [23]:
#a)
def f(x):
    return x*(x-1)

def derivative ( delta):
    der= ( f(1+delta) - f(1))/ delta 
    return der
    
derivative(10**(-2))
#analiticamente viene fuori 1.01, questo perche python usa float a
#doppia precisione ( 64 bit) , numeri come 0.01 ed 1.01 non 
#sono rappresentati esattamente, inducendo un piccolo errore aggiuntivo
#(errore di floating point) 

1.010000000000001

In [26]:
#b)
def f(x):
    return x*(x-1)

def derivative ( delta):
    der= ( f(1+delta) - f(1))/ delta 
    return der

print ( f' for delta=10^-4 :{derivative(10**(-4))}')
print ( f' for delta=10^-6 :{derivative(10**(-6))}')
print ( f' for delta=10^-8 :{derivative(10**(-8))}')
print ( f' for delta=10^(-10) :{derivative(10**(-10))}')
print ( f' for delta=10^(-12) :{derivative(10**(-12))}')
print ( f' for delta=10^(-14) :{derivative(10**(-14))}')

#Man mano che il valore di \delta diminuisce, l’approssimazione 
#numerica della derivata migliora inizialmente, avvicinandosi al 
#valore analitico. Questo accade perché la definizione di derivata
#è valida nel limite \delta \to 0, quindi valori più piccoli di 
#\delta riducono l’errore di discretizzazione. Tuttavia, quando 
#\delta diventa troppo piccolo, l’accuratezza inizia a peggiorare 
#a causa degli errori di rappresentazione dei numeri in macchina 
#(floating point). In questo caso, la differenza tra f(1+\delta)
#e f(1) coinvolge due numeri molto simili, e la sottrazione numerica
#amplifica l’errore di arrotondamento. Di conseguenza, l’errore 
#totale mostra un comportamento “a U”: migliora al diminuire di
#\delta fino a un certo punto, per poi aumentare di nuovo quando 
#\delta diventa troppo piccolo.

 for delta=10^-4 :1.0000999999998899
 for delta=10^-6 :1.0000009999177333
 for delta=10^-8 :1.0000000039225287
 for delta=10^(-10) :1.000000082840371
 for delta=10^(-12) :1.0000889005833413
 for delta=10^(-14) :0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [27]:
#a) 
import numpy as np

N = 100
h = 2 / N
x = np.linspace(-1 + h/2, 1 - h/2, N)  # aggiungi un piccolo h/2 per non perdere precisione, altrimenti avresti in y = 1-1, perche i limiti sono quelli 
y = np.sqrt(1 - x**2)
I_approx = np.sum(y * h)

print("Approximate integral with N=100:", I_approx)
print("True value (π/2):", np.pi / 2)
print("Error:", np.abs(I_approx - np.pi/2))

#b) da finire 



Approximate integral with N=100: 1.5712827762297956
True value (π/2): 1.5707963267948966
Error: 0.0004864494348990611
